In [1]:
#ONLY RUN THIS FILE ONCE! get_all_objects() is slow... luckily I've saved the resulting dataframe as a .pkl

import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import nltk
from nltk.stem import PorterStemmer as ps
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
stemmer = ps()
from nltk.stem.cistem import Cistem
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
import uuid
from concurrent.futures import ThreadPoolExecutor, as_completed


In [2]:
def api_call(x):
    session = requests.Session()
    data_list = []
    parameters = {"items_per_page": 25,
                      "page": x}
    response = session.get("https://risdmuseum.org/api/v1/collection", params=parameters).json()
    for item in response:
        data_list.append(item)
    return data_list 
    
        

    

In [ ]:
def runner(function, var):
    giant_list = []
    threads= []
    with ThreadPoolExecutor(max_workers=20) as executor:
        for x in range(0, var):
            threads.append(executor.submit(function, x))
            
        for task in as_completed(threads):
            giant_list.append(task.result())
    return giant_list

In [4]:
data_list = runner(api_call, 3289)

In [19]:
def dict_to_df(a_dict):
    new_df = pd.json_normalize(a_dict)
    new_df['makers'] = new_df['makers'].astype(str)
    filtered_df = new_df[new_df['makers'].str.contains("'name': 'Unknown'")]
    return filtered_df

In [24]:
def dict_runner(function, var):
    df = pd.DataFrame()
    threads= []
    with ThreadPoolExecutor(max_workers=20) as executor:
        for a_dict in var:
            threads.append(executor.submit(function, a_dict))
            
        for task in as_completed(threads):
            df = df.append(task.result())
    return df

In [25]:
object_df = dict_runner(dict_to_df, data_list)

In [27]:
object_df.to_pickle("./unknown_makers_FINAL.pkl")